# Titanic example

This notebook illustrate a toy example, with three passengers, and uses a squared distance classifier, inspired by Machine Learning with Quantum Computers by Schuld and Petruccione, to predict if a passenger will survive the 2021 Titanic disaster.

Two of the three passengers are the training set.  One survived and one died. The aim is to predict the fate of the third passenger in the mini-test set.  

Data is given for all three passengers consisting of a cabin number, assumed to be between 1 and 2,500, and a ticket price, assumed to be between £1 and £10,000, stored in a vector $\bf{x_m}$ for the training data, and $\bf{x}$ for the test data.  A nearest neighbour classifier is used to classify the third passenger, with $$p(y=1|\bf{x}) = \frac{1}{\chi} \frac{1}{M_1} \sum_{m|y^m = 1}\left( 1 - \frac{1}{c}\|\bf{x} - \bf{x^m} \|^2\right)$$

where $M_1$ is the sum over all training inputs labeled with $y_m$ = 1, $c$ is an arbitary constant, and $\chi$ is a normalisation factor to ensure $p(y = 0|\bf{x}) + p(y = 1|\bf{x}) = 1$

Import modules needed:

In [ ]:
from pathlib import Path
import numpy as np
import math
import pennylane as qml

HOME_DIR = '..'
BASE_DIR = Path(HOME_DIR)

import sys
sys.path.append(HOME_DIR)

from config.config import DATA
PROJECT = '01_titanic'
FOLDER = 'processed'
FILE = 'processed_data.csv'

from src.modules.data_helper_functions import (read_csv, 
                                              find_gamma_m, 
                                              normalise,
                                              find_norm,
                                              pre_process_feature_vector,
                                              prepare_quantum_feature_vector,
                                              normalise_feature_vector,
                                              )

from src.modules.graph_functions import plot_simple_scatter

Load the data

In [ ]:
file_path = BASE_DIR.joinpath(DATA).joinpath(PROJECT).joinpath(FOLDER).joinpath(FILE)
print(f'Data will be loaded from {file_path}')
data = read_csv(file_path)
print(f'The raw data is:')
for items in data:
    print(items)

Clean and print the data:

In [ ]:

labels, x1, x2, y = [], [], [], []

for row in data:
    labels.append(row['passenger'])
    x1.append(float(row['price']))
    x2.append(float(row['cabin']))
    y_val = (row['survived'])
    if y_val:
        y.append(int(y_val))
    else:
        y.append(y_val)

print(f'labels = {labels}')
print(f'x1 = {x1}')
print(f'x2 = {x2}')
print(f'y - classification result {y}')

Plot data

In [ ]:
plot_simple_scatter(x1, x2, labels, y)

Calculate the square distance classifier:

First find the test data:

In [ ]:
for i, items in enumerate(y):
    if items == '':
        x = np.array([x1[i],x2[i]])
if x.shape != (2,):
    raise Exception(f'x,shape should be (2,), is {x.shape}')
print(f'The test point is {x}')

Calculate $p(y = 0|\bf{x})$ and $p(y = 1|\bf{x})$, and print results

In [ ]:
M0, M1, p0, p1 = 0, 0, 0, 0
for i, item in enumerate(y):
    x_m = np.array([x1[i],x2[i]])
    if item == 1:
        M1 += 1
        p1 += find_gamma_m(x, x_m)
        print(f'For point {x_m} gamma_m with a passenger who survived at point {x} is {p1:.3f}.')
    elif item == 0:
        M0 += 1
        p0 += find_gamma_m(x, x_m)
        print(f'For point {x_m} gamma_m with a passenger who died at point {x} is {p0:.3f}.')
    elif item != '':
        raise Exception('Value of y is {item} which is not allowed') 
    
p0, p1 = p0/M0, p1/M1 # find average value
#normalise
chi = p0 + p1
p0, p1 = p0/chi, p1/chi

print(f'The probability that the test passenger dies is {p0:.1%}.')
print(f'The probability that the test passenger survives is {p1:.1%}.')

if p1 > p0:
    print('The classifier predicts survival!')
else:
    print('The classifier predicts death!')

Normalise, print and plot

In [ ]:
x1, x2 = normalise(x1, x2)
print(f'x1={[f'{v:.3f}' for v in x1]}\r')
print(f'x2={[f'{v:.3f}' for v in x2]}\r')
print(f'y={y}')
plot_simple_scatter(x1, x2, labels, y)

In [ ]:
x1, x2, y

Prepare the data ready to load into a quantum computer
- Add an extra copy of the features of Passsenger 3, and 
- tidy up y to be integer

In [ ]:
x1, x2, y = pre_process_feature_vector(x1, x2, y)


Prepare a complete feature vector

In [ ]:
alpha = prepare_quantum_feature_vector(x1, x2, y)

In [ ]:
type(alpha)

In [ ]:
"""alpha_norm = []
norm = float(find_norm(alpha))
print(f'{norm=}')
for items in alpha:
    alpha_norm.append(float(items)/norm)
print(alpha)
print(f'alpha_norm={[f'{v:.3f}' for v in alpha_norm]}\r')"""
alpha_norm = normalise_feature_vector(alpha)

In [ ]:
type (alpha_norm)

In [ ]:
norm = float(find_norm(alpha_norm))
print(f'{norm=}')

In [ ]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=4)

# Example normalized feature vector (length 16)
features = alpha_norm # Uniform distribution, already normalized

def prepare_amplitudes(features, wires):
    """Recursively prepare amplitudes using rotations and controlled rotations."""
    n = len(wires)
    if n == 1:
        # Single qubit: apply RY rotation
        theta = 2 * np.arccos(features[0] / np.sqrt(features[0]**2 + features[1]**2))
        qml.RY(theta, wires=wires[0])
    else:
        # Split amplitudes into two halves
        half = len(features) // 2
        norm_left = np.sqrt(np.sum(features[:half]**2))
        norm_right = np.sqrt(np.sum(features[half:]**2))

        # Compute rotation angle for the first qubit
        theta = 2 * np.arccos(norm_left / np.sqrt(norm_left**2 + norm_right**2))
        qml.RY(theta, wires=wires[0])

        # Apply controlled rotations recursively
        qml.ctrl(prepare_amplitudes, control=wires[0])(features[:half], wires[1:])
        qml.ctrl(prepare_amplitudes, control=wires[0], control_values=[1])(features[half:], wires[1:])

@qml.qnode(dev)
def manual_amplitude_embedding():
    prepare_amplitudes(features, wires=list(range(4)))
    return qml.state()

#print(qml.draw(manual_amplitude_embedding)())


In [ ]:
#qml.draw_mpl(expanded_circuit)(alpha_norm)